In [43]:
import pandas as pd
import numpy as np
import Levenshtein
import re
import unidecode 

data = pd.read_csv('Banco de Súmulas - Sumulas.csv')

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_14500\2494190213.py:8: DtypeWarning: Columns (8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Banco de Súmulas - Sumulas.csv')


In [44]:
# keep only single classes
# classes = ['IQFOIL Fem.', 'Formula Kite Fem.', 'IQFOIL Masc.','Formula Kite Masc.', 'Ilca 7', 'Ilca 6']
classes = ['IQFOIL Fem.', 'Formula Kite Fem.', 'IQFOIL Masc.','Formula Kite Masc.']
data = data[data['Classe Vela'].isin(classes)]

# upper case all "Nome Competidor" 
data['Nome Competidor'] = data['Nome Competidor'].str.upper()

# unidecode all "Nome Competidor"
data['Nome Competidor'] = data['Nome Competidor'].apply(unidecode.unidecode)

# replace double spaces with single space
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')

# get unique names
names = data['Nome Competidor'].unique()
len(names)

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_14500\2066453530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Nome Competidor'] = data['Nome Competidor'].str.upper()
C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_14500\2066453530.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Nome Competidor'] = data['Nome Competidor'].apply(unidecode.unidecode)
C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_14500\2066453530.py:13: SettingWithCopyWarning: 
A value is trying to be set on a

955

In [45]:
# fix mixed up names function
def are_names_same(str1, str2):
    # Split the strings into words, remove spaces, and sort the lists of words
    name1 = sorted(str1.split())
    name2 = sorted(str2.split())
    
    # Check if the sorted lists of words are equal
    return name1 == name2

In [46]:
# copy names to a list
names_list = names.tolist()

# create a list with people with mixed up names, adding each pair as a sublist
mixed_up_names = []
for name1 in names_list:
    for name2 in names_list:
        if are_names_same(name1, name2) and name1 != name2:
            mixed_up_names.append([name1, name2])
            # avoid inserting the same pair by substituting name1 with name2 in the names list
            names_list = np.where(names_list == name1, name2, names_list)

In [47]:
# levenshtein distance

# create df where rows and columns are the names and the values are the levenshtein distance
names_comparison = np.array(np.meshgrid(names, names)).T.reshape(-1,2)

# create a third column with the levenshtein distance
names_comparison = pd.DataFrame(names_comparison, columns=['name', 'other_names'])
names_comparison['levenshtein_distance'] = names_comparison.apply(lambda row: Levenshtein.distance(row['name'], row['other_names']), axis=1)

In [48]:
# set thresholds
ld1 = names_comparison[names_comparison['levenshtein_distance'] == 1]
ld2 = names_comparison[names_comparison['levenshtein_distance'] == 2]
ld3 = names_comparison[names_comparison['levenshtein_distance'] == 3]
ld4 = names_comparison[names_comparison['levenshtein_distance'] == 4]
ld5 = names_comparison[names_comparison['levenshtein_distance'] == 5]

In [49]:
# create dict with the names that are similar (dont insert same pair twice)
similar_names = {}

for ld in [ld1, ld2, ld3, ld4, ld5]:
    for _, row in ld.iterrows():
        # if the name is already in the dict
        if row['name'] in similar_names:
            # if the other name is not in the list, add it
            if row['other_names'] not in similar_names[row['name']]:
                similar_names[row['name']].append(row['other_names'])
            # if it is, do nothing
            else:
                pass

        # if the name is not in the dict and the other name is
        elif row['other_names'] in similar_names.values():
            # if the name isnt on other_name key, add it
            if row['name'] not in similar_names[row['other_names']]:
                similar_names[row['other_names']].append(row['name'])
            # if it is, do nothing
            else:
                pass
            
        # if none of the names are in the dict, add them as a new key
        else:
            # check which appears the most in the data and add that one as the key
            if data[data['Nome Competidor'] == row['name']].shape[0] > data[data['Nome Competidor'] == row['other_names']].shape[0]:
                similar_names[row['name']] = [row['other_names']]
            else:
                similar_names[row['other_names']] = [row['name']]

In [50]:
# if two keys receive True from the are_names_same function, 
# add the second key and value to the first key's value list and delete the second key

# create a list for the keys that will be deleted
keys_to_delete = []

for key1 in similar_names:
    for key2 in similar_names:
        # if the keys are the same, skip
        if key1 == key2:
            pass
        elif are_names_same(key1, key2) and (key2 not in keys_to_delete) and (key1 not in keys_to_delete):
            print(key1, key2)
            # add key2 value to key1 value
            similar_names[key1].extend(similar_names[key2])
            # add key 2 to key 1 value
            similar_names[key1].append(key2)
            # add key 2 to keys to delete
            keys_to_delete.append(key2)

print(keys_to_delete)

# delete the keys that are in the keys_to_delete list
for key in keys_to_delete:
    del similar_names[key]

WAN LI LI WAN
['LI WAN']


In [51]:
# for each sublist in mixed_up_names, check if the mixed_up_names[0] is in similar_names as a key
# if it is, add mixed_up_names[1] to the value list

for pair in mixed_up_names:
    if pair[0] in similar_names:
        # append pair[1] to key pair[0]
        similar_names[pair[0]].append(pair[1])
    elif pair[0] in similar_names.values():
        # append pair[1] to the key that has pair[0] as a value
        for key in similar_names:
            if pair[0] in similar_names[key]:
                similar_names[key].append(pair[1])
    elif pair[1] in similar_names:
        # append pair[0] to key pair[1]
        similar_names[pair[1]].append(pair[0])
    elif pair[1] in similar_names.values():
        # append pair[0] to the key that has pair[1] as a value
        for key in similar_names:
            if pair[1] in similar_names[key]:
                similar_names[key].append(pair[0])
    else:
        # add pair[0] and pair[1] as a new key and value
        similar_names[pair[0]] = [pair[1]]

In [53]:
# if a key has repeated values, delete one of them
for key in similar_names:
    similar_names[key] = list(set(similar_names[key]))
similar_names

{'LUKAS WALTON-KEIM': ['LUKAS WALTON- KEIM'],
 'LUCAS PES FONSECA': ['LUCAS PE FONSECA'],
 'SIRIWIT PRANGSRI': ['SIRAWIT PRANGSRI'],
 'GEORGE MANEV': ['GEORGI MANEV'],
 'MAYA ASHKENAZI': ['MAYA ASSHKENAZI', 'ASHKENAZI MAYA'],
 'MARIE-EVE MAYRAND': ['MARIE EVE MAYRAND'],
 'JAN VOSTER': ['VOSTER JAN', 'JAN VOESTER'],
 'SEBASTIAN KOERDEL': ['SEBASTIAN KORDEL', 'KOERDEL SEBASTIAN'],
 'MATTHEW BARTON': ['MATHEW BARTON'],
 'RADOSLAW FURMANSKI': ['RADOSLAW FUMANSKI', 'FURMANSKI RADOSLAW'],
 'BYRON KOKKALANIS': ['VYRON KOKKALANIS'],
 'ANTONINO CANGEMI': ['ANTONIO CANGEMI'],
 'FREDERIC RAMSGAARD ORUM': ['FREDERIC RAMSGAARD OERUM'],
 'NOCETI-KLEPACKA ZOFIA': ['NOCETI KLEPACKA ZOFIA'],
 'LEE TAEHOON': ['LEE TAE HOON', 'TAEHOON LEE'],
 'DIMITRI MARAMENIDES': ['DIMITRIS MARAMANIDES'],
 'JINGYUE CHEN': ['JINGLE CHEN', 'CHEN JINGYUE'],
 'WAN LI': ['WANG SI', 'BI KUN', 'KUN BI', 'LI WAN'],
 'SI WANG': ['WANG SI', 'LI WAN'],
 'TUVA OPPEDAL': ['THEA OPPEDAL'],
 'JOSH ARMIT': ['JOSHUA ARMIT'],
 'MARTINS 

In [61]:
# reorder similar_names dict by alphabetical order of the keys
similar_names = dict(sorted(similar_names.items(), key=lambda item: item[0])) 

In [54]:
# output similar_names to json
import json

with open('similar_names_ld.json', 'w') as fp:
    json.dump(similar_names, fp)

In [55]:
# read json to dict
with open('similar_names_ld.json', 'r') as fp:
    similar_names_dict = json.load(fp)

# for each key, substitute the values by the key in the data
for key in similar_names_dict:
    for value in similar_names_dict[key]:
        data['Nome Competidor'] = np.where(data['Nome Competidor'] == value, key, data['Nome Competidor'])

In [58]:
# show data where Nome Competidor is MAXIME NOCHER
data[data['Nome Competidor'] == 'MAX NOCHER']

,ID Resultado,ID Competidor,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Descarte,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição
